MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from [Francesca Lazzeri Energy Demand Forecast Workbench workshop](https://github.com/FrancescaLazzeri/EnergyDemandForecastWorkbench).

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva


# Data preparation
Run this notebook to prepare the data for modelling and registering prepared datasets.

**Important prerequisites**: 
- Change the kernel to Python 3.8.10...('mlops_train':conda) You can do this from the *Kernel* menu under *Change kernel*.

Run each cell of this notebook to perform the following steps:
- Import the data from csv files and merge the two datasets.
- The data is cleaned by filling gaps in the time series and handling missing values.
- The data is explored through visualisation.
- Features for the forecasting models are computed.
- Data is split into training and test sets.
- Connect to Azure Machine learning workspace and register processed, train, test datasets

In [ ]:
%matplotlib inline
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from azureml.core import Workspace, Dataset
from pandas.plotting import autocorrelation_plot
from scipy import stats

### Load energy demand data and merge with weather data

In [ ]:
WORKDIR = os.getcwd()
DATADIR = os.path.join(WORKDIR, 'datasets')

In [ ]:
demand_raw = pd.read_csv(os.path.join(DATADIR,'nyc_demand_raw.csv'), parse_dates=['timeStamp'])
weather_raw = pd.read_csv(os.path.join(DATADIR,'nyc_weather_raw.csv'), parse_dates=['timeStamp'])
demand = pd.merge(demand_raw, weather_raw, on=['timeStamp'], how='outer')
demand['demand'] = demand['demand'].astype(float)
demand.head()

In [ ]:
plt_df = demand.loc[(demand.timeStamp>'2016-07-01') & (demand.timeStamp<='2016-07-07')]
plt.plot(plt_df['timeStamp'], plt_df['demand'])
plt.title('New York City power demand over one week in July 2017')
plt.xticks(rotation=45)
plt.show()

### Fill gaps in the time series

Some periods in the time series are missing. This occurs if the period was missing in both the original demand and weather datasets. To identify these gaps, first we create an index of time periods that we would *expect* to be in the time series. There should be one record for every hour between the minimum and maximum datetimes in our dataset.

In [ ]:
min_time = min(demand['timeStamp'])
min_time

In [ ]:
max_time = max(demand['timeStamp'])
max_time

In [ ]:
dt_idx = pd.date_range(min_time, max_time, freq='H')
dt_idx

Now we index the dataframe according to this datetime index to insert missing records into the time series:

In [ ]:
demand.index = demand['timeStamp']
demand = demand.reindex(dt_idx)

Inserted missing records will have NaN/NaT values for all columns:

In [ ]:
demand[demand.isnull().all(axis=1)]

Now that there are no missing periods in the time series, we can start handling missing values by filling as many many as possible. Firstly, count the number of missing values in each column:

In [ ]:
demand.isnull().sum()

Missing timeStamp can be filled from the dataframe index:

In [ ]:
demand.loc[demand.isnull().all(axis=1), 'timeStamp'] = demand.loc[demand.isnull().all(axis=1)].index

For the other columns, we can fill many missing values by interpolating between the two closest non-missing values. Here, we use a quadratic function and set a limit of 6. This limit means that if more than 6 missing values occur consecutively, the missing values are not interpolated over and they remain missing. This is to avoid spurious interpolation between very distant time periods.

In [ ]:
demand = demand.interpolate(limit=6, method='linear')

Fill missing precip values with common value of 0:

In [ ]:
precip_mode = np.ndarray.item(stats.mode(demand['precip']).mode)
demand['precip'] = demand['precip'].fillna(precip_mode)

In [ ]:
demand.isnull().sum()

The number of missing values has now been greatly reduced. Records containing the remaining missing values will be removed later after model features have been created.

### Explore the data

By visualising the data, we can gain some intuition as to what kind of features could be helpful to the model.

In [ ]:
plt.hist(demand['demand'].dropna(), bins=100)
plt.title('Demand distribution')
plt.show()

In [ ]:
plt_df = demand.copy().loc[(demand['timeStamp']>='2016-01-01') & (demand['timeStamp']<'2017-01-01'), ]
plt.plot(plt_df['timeStamp'], plt_df['demand'], markersize=1)
plt.title('Hourly demand in 2016')
plt.ylabel('demand')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.hist(demand['temp'].dropna(), bins=100)
plt.title('Temperature distribution')
plt.show()

In [ ]:
plt.plot(demand['temp'], demand['demand'], 'ro', markersize=1)
plt.title('Demand vs temperature')
plt.xlabel('temp')
plt.ylabel('demand')
plt.show()

The autocorrelation plot below shows the extent to which the demand variable correlates with itself at different intervals (lags). This plot shows that demand is highly autocorrelated over the closest 6 hour period.

In [ ]:
autocorrelation_plot(demand['demand'].dropna())
plt.xlim(0,24)
plt.title('Auto-correlation of hourly demand over a 24 hour period')
plt.show()

### Compute features for forecasting models

After exploring the data, it is clear that the energy demand follows seasonal trends, with daily, weekly and annual periodicity. We will create features that encode this information. First, we compute time driven features based on timeStamp. Note for dayofweek, Monday=0 and Sunday=6.

In [ ]:
demand_features = demand.copy()

In [ ]:
demand_features['hour'] = demand_features.timeStamp.dt.hour
demand_features['month'] = demand_features.timeStamp.dt.month-1
demand_features['dayofweek'] = demand_features.timeStamp.dt.dayofweek

Compute lagged demand features

In [ ]:
def generate_lagged_features(df, var, max_lag):
    for t in range(1, max_lag+1):
        df[var+'_lag'+str(t)] = df[var].shift(t, freq='1H')

In [ ]:
generate_lagged_features(demand_features, 'temp', 6)
generate_lagged_features(demand_features, 'demand', 6)

In [ ]:
demand_features.head()

### Final data cleaning and write out training and test datasets

Count remaining null values.

In [ ]:
demand_features.isnull().sum()

Count number of rows with any null values

In [ ]:
demand_features.loc[demand_features.isnull().any(axis=1), ].shape[0]

This is a very small proportion of the overall dataset so can be safely dropped.

In [ ]:
demand_features.dropna(how='any', inplace=True)

In [ ]:
demand_features.to_csv(os.path.join(DATADIR, 'nyc_demand_processed.csv'), index=False)

Split data into training and test datasets. All data after 1st July 2016 is reserved for the test set.

In [ ]:
train, test = (demand_features.loc[demand_features['timeStamp']<'2016-07-01'], demand_features.loc[demand_features['timeStamp']>='2016-07-01'])
train.to_csv(os.path.join(DATADIR, 'nyc_demand_train.csv'), float_format='%.4f', index=False)
test.to_csv(os.path.join(DATADIR, 'nyc_demand_test.csv'), float_format='%.4f', index=False)

### Register datasets to the workspace

In [ ]:
# get the workspace from config.json
ws = Workspace.from_config()
# get the datastore to upload our data
datastore = ws.get_default_datastore()

In [ ]:
datastore.upload(src_dir='datasets', target_path='data')

In [ ]:
processed_dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/nyc_demand_processed.csv'))

In [ ]:
# preview the first 3 rows of the dataset from datastore
processed_dataset.take(3).to_pandas_dataframe()

In [ ]:
processed_ds = processed_dataset.register(workspace=ws, name="processed_nyc_demand_data", description="processed New York energy demand data")

In [ ]:
train_dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/nyc_demand_train.csv'))
test_dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/nyc_demand_test.csv'))

In [ ]:
train_dataset.take(3).to_pandas_dataframe()

In [ ]:
test_dataset.take(3).to_pandas_dataframe()

In [ ]:
train_ds = train_dataset.register(workspace=ws, name="train_nyc_demand_data", description="processed New York energy demand data for training")
test_ds = test_dataset.register(workspace=ws, name="test_nyc_demand_data", description="processed New York energy demand data for testing")